In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.model_selection
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
import plotly.graph_objs as go
import statistics
import scipy.stats as stats
import math
import re

In [193]:
data = pd.read_csv('zameen_data.csv')

In [194]:
data = data[data["purpose"]=="For Sale"]

In [195]:
len(data)

35829

In [196]:
area_size = data['area'].str.split(expand=True)
area_size.columns = ["area size","area type"]
data = pd.concat([data, area_size], axis=1)

In [197]:
data['area size']= data['area size'].astype(float)

In [198]:
data.reset_index(inplace=True, drop=True)

In [199]:
data["area in sq_ft"] = 0

for i in range(len(data['area size'])):
    if(data['area type'][i] == 'Marla'):
        data['area in sq_ft'][i] = data['area size'][i]*272.25
    else:
        data['area in sq_ft'][i] = data['area size'][i]*5445  

C:\Users\Bilawal Aka Bilal\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Bilawal Aka Bilal\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [182]:
data.head()

,property_id,location_id,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,agency,agent,area size,area type,area in sq_ft
0,237062,3325,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2,NaN,NaN,4.0,Marla,1089
1,346905,3236,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3,NaN,NaN,5.6,Marla,1524
2,386513,764,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5,NaN,NaN,8.0,Marla,2178
3,656161,340,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,2 Kanal,For Sale,4,NaN,NaN,2.0,Kanal,10890
4,841645,3226,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3,Easy Property,Muhammad Junaid Ceo Muhammad Shahid Director,8.0,Marla,2178


In [200]:
data = data.drop(['latitude','longitude','purpose','city','province_name','agency','agent','property_id','location_id','area type','area size','area'], axis='columns')

In [201]:
data.reset_index(drop=True, inplace=True)

In [185]:
# Handling/Removing Ouliers
for i in range(len(data["price"])):
    if(data["price"][i] > 400000000):
        data.drop(index = [i], inplace = True)


In [186]:
data.reset_index(drop=True, inplace=True)

In [202]:
data = pd.get_dummies(data, columns = ['property_type','location'], drop_first=True)

In [204]:
labels = data["price"]
train = data.drop( columns=["price"])
x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.35, random_state = 12)

In [189]:
data.head()

,price,baths,bedrooms,area in sq_ft,property_type_Flat,property_type_House,property_type_Lower Portion,property_type_Penthouse,property_type_Room,property_type_Upper Portion,...,location_Zamzama,location_Zaraj Housing Scheme,location_Zeeshan Colony,location_Zero Point,location_Zia Colony,location_Zone 5,location_Zubaida Park,location_Zubair Colony,location_Zulfiqar Colony,location_circular road
0,10000000,2,2,1089,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6900000,3,3,1524,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16500000,6,5,2178,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,43500000,4,4,10890,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7000000,3,3,2178,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [205]:
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators = 150, max_depth = 15)
est.fit(x_train, y_train)
est.score(x_train, y_train)*100

93.61832383171202

In [ ]:
check_real_predict = {"y_test":y_test,"y_pred":abs(y_pred),"ERROR":abs((y_test - y_pred)/y_test)*100}
pd.DataFrame(check_real_prdict)